<a href="https://colab.research.google.com/github/ghizlanebo/nltk/blob/main/LDA_BOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.graph_objs as go
import gensim
from gensim import corpora, models, similarities
import logging
import tempfile
from nltk.corpus import stopwords
from string import punctuation
from collections import OrderedDict
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

init_notebook_mode(connected=True) #do not miss this line

import warnings
warnings.filterwarnings("ignore")

In [2]:
papers = pd.read_excel('/content/drive/MyDrive/REC_CG_19-21.xlsx')
# Print head
papers.head()

,Rec_DateEnvoi,Rec_Id,Typeaccueil,MotifTypeLibFr,Motif_Lib,Rec_EtatType_Lib,Rec_DateEtat,descri
0,2019-01-21 14:27:00,1,Visite,Réclamation,Remises en paiement,Traité,2019-06-17 16:29:00,- Concerné(s): MOHAMED & OUMAIMA - Situation...
1,2020-05-29 10:37:00,2,Téléphone,Demande de renseignement,Allocations Familiales,Traité,2020-05-29 10:44:00,L’intéressé a réclame date de naissance de en...
2,2019-11-27 11:07:00,3,Téléphone,Demande de renseignement,Précomptes,Traité,2019-11-27 11:08:00,"672,00DH] [ATD DEFENSE"
3,2020-12-08 10:12:00,4,Visite,Nonaffiché,Autre,Traité,2020-12-08 10:12:00,Attestation pension
4,2020-12-08 10:08:00,5,Téléphone,Demande de renseignement,Pension principale,Traité,2020-12-08 10:11:00,"RAPPEL : [ 9018,52DH] [PREMIER ARRERAGE"


In [3]:
# Remove the columns
papers = papers.drop(columns=['Rec_Id', 'Rec_DateEtat'], axis=1).sample(100)
# Print out the first rows of papers
papers.head()

,Rec_DateEnvoi,Typeaccueil,MotifTypeLibFr,Motif_Lib,Rec_EtatType_Lib,descri
115456,2020-09-10 11:46:00,Téléphone,Demande de renseignement,Pension principale,Traité,dossier pour liquidation
263034,2019-08-08 17:01:00,Téléphone,Demande de renseignement,Pension principale,Traité,appel coupè.
101469,2020-08-05 10:26:00,Téléphone,Demande de renseignement,Réversion,Traité,problème sur ged
83349,2020-06-24 09:25:00,Téléphone,Demande de renseignement,Pension principale,Traité,Traité pour le mois 8
104020,2020-08-12 09:13:00,Téléphone,Réclamation,Derniers arrérage,Traité,Statut :Dossier simple ouvert et non encore mi...


In [4]:
papers['descri'].head()

115456                             dossier pour liquidation
263034                                         appel coupè.
101469                                     problème sur ged
83349                                 Traité pour le mois 8
104020    Statut :Dossier simple ouvert et non encore mi...
Name: descri, dtype: object

In [5]:
l=papers['descri'].tolist()

In [6]:
corpus=[]
a=[]
for i in range(len(l)):
        a=l[i]
        corpus.append(a)
        
corpus[0:10]

['dossier pour liquidation',
 'appel coupè.',
 'problème sur ged',
 'Traité pour le mois 8',
 'Statut :Dossier simple ouvert et non encore mise en paiement, Concerné(s) :Mandataire (CHAMSI SOUAD), Documents :Demande DA + Procuration héritiers + RIB + CIN, Dossier de recherche :DCP, Clé de recherche :7069852, Délai de retard :10 mois, Réclamations répétitives :Oui',
 'simulation',
 'DEMANDE D’INF',
 'Motif de radiation ...:  Limite d’âge                                      Date de radiation ....:  31/08/2020  11876.95',
 'DDE. D INFOS.',
 'Dépôt demande d’attestation de main levée']

In [7]:
my_stop_words= ['cmr','bonjour','merci','comme', 'si', 'faire', 'cette', 'aussi', 'ainsi', 'toutes','beaucoup', 'faut', 'falloir',
              'cela', 'va', 'aller', 'là', 'ça','peux', 'pouvoir', 'quand', 'encore', 'donc', 'toujours', 
              'où','quelques', 'autre', 'autres', 'alors', 'entre', 'car', 'après','avant','depuis', 'également', 'un','trois', 'quatre', 'avoir', 'bon', 
              'première', 'premier','deuxième', 'niveau', 'face', 'contre', 'journée', 'début','jours', 
              'journée', 'année', 'jamais', 'lu', 'lire','selon', 'hui', 'aujourd', 'grand','dire', 'dit',
              'dis', 'cet','cette','nouveau','nouvelle','mis','mettre','chez','1','2','3','4','5','6','7','8','9','0',']','[','(',')','#','$',',',':','/','NaN','ddr','br','á','ã', 'í','Ê','î','ê','Ú','Ï','nan','dde','de','non','oui','2019','2020','le','encore']

In [8]:
from nltk.corpus import stopwords
from string import punctuation
import nltk
nltk.download('stopwords')
from gensim import corpora
stoplist = stopwords.words('french') + list(punctuation) + my_stop_words

texts = [[word for word in str(document).lower().split() if word not in stoplist] for document in corpus]

dictionary = corpora.Dictionary(texts)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
corpus = [dictionary.doc2bow(text) for text in texts]


In [10]:
total_topics = 14

In [11]:
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=total_topics)


In [12]:
corpus_lda = lda[corpus]

In [13]:
lda.show_topics(total_topics,14)

[(0,
  '0.042*"dossier" + 0.030*"affecté" + 0.024*"recherche" + 0.015*"reversion" + 0.015*"no" + 0.015*"bonjour," + 0.015*"12/06/2019," + 0.015*"4013819" + 0.015*"concedé" + 0.015*"arrete:" + 0.015*"ged" + 0.015*"enregistrer" + 0.014*"rib" + 0.014*"cin,"'),
 (1,
  '0.054*"retraite" + 0.054*"cin" + 0.028*"inchaallh" + 0.028*"mars" + 0.028*"situation" + 0.028*"m’informer" + 0.028*"assistance" + 0.028*"2022" + 0.028*"prière" + 0.028*"rectifiée" + 0.028*"k58583" + 0.028*"infos." + 0.028*"dde." + 0.002*"d’infos"'),
 (2,
  '0.041*"&#9474;" + 0.041*"2300,00dh]" + 0.041*"dossier" + 0.041*"a" + 0.021*"rap" + 0.021*"[rappel" + 0.021*"ret" + 0.021*"arrerages" + 0.021*"annule" + 0.021*"[premiers" + 0.021*"enregistré" + 0.021*"délivré" + 0.021*"familiales" + 0.021*"allocations"'),
 (3,
  '0.037*"d’infos" + 0.024*"mois" + 0.019*"traité" + 0.019*"répétitives:oui" + 0.019*"statut:non" + 0.019*"réclamations" + 0.019*"concerné(s):lui-même" + 0.019*"retard:+24" + 0.019*":révision" + 0.019*"documents" + 0

In [14]:
from collections import OrderedDict

data_lda = {i: OrderedDict(lda.show_topic(i,25)) for i in range(total_topics)}

In [15]:
df_lda = pd.DataFrame(data_lda)
df_lda = df_lda.fillna(0).T
print(df_lda.shape)

(14, 219)


In [16]:
df_lda

,dossier,affecté,recherche,reversion,"bonjour,",no,"12/06/2019,",4013819,concedé,arrete:,ged,enregistrer,rib,"cin,",procuration,da,concerné(s),:mandataire,retard,"paiement,",simple,":dcp,",:10,mise,héritiers,retraite,cin,inchaallh,situation,mars,m’informer,assistance,2022,prière,rectifiée,k58583,infos.,dde.,d’infos,simulation,...,"rabat,",cordialement,araar,suivante,bp,débuté,echelon1,01/01/2011..je,nécessaire.,chef,ingenieur,principal,vouloir,grade,d’âge,échéances,paiement,limite,respects,[arret,suspension+arret,suspension,"135090,60dh]",liquidé,01/05/2019,reclamation,recherche:,&#9474;date,&#9474;&#9474;,clé,e,retard:,lieud,k77673,dcpc,statut:,r77376,modification,liquiation,service
0,0.042131,0.029819,0.0243,0.015424,0.015424,0.015424,0.015424,0.015424,0.015424,0.015424,0.015424,0.015424,0.01383,0.013724,0.013634,0.013617,0.013588,0.013381,0.013298,0.013119,0.013096,0.013085,0.012999,0.012937,0.012839,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,0.001855,0.001855,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001855,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.053803,0.053803,0.027829,0.027829,0.027829,0.027829,0.027829,0.027829,0.027829,0.027829,0.027829,0.027829,0.027829,0.001855,0.001855,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2,0.041018,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021216,0.001414,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019362,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037434,0.019362,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
4,0.000000,0.000000,0.0000,0.000000,0.003845,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.048293,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

In [22]:
print('\nPerplexity: ', lda.log_perplexity(corpus_lda,total_docs=100000))


Perplexity:  -8.790428512816389


In [23]:
print('\nPerplexity: ', lda.log_perplexity(corpus_lda,total_docs=10000))


Perplexity:  -8.865699680948126


In [24]:
print('\nPerplexity: ', lda.log_perplexity(corpus_lda,total_docs=300000))


Perplexity:  -8.778232784513943


In [25]:
print('\nPerplexity: ', lda.log_perplexity(corpus_lda,total_docs=3000))


Perplexity:  -9.080840451869081


In [26]:
from gensim.models.coherencemodel import CoherenceModel


In [27]:
coherence_model_lda = CoherenceModel(model=lda, texts=texts, dictionary=dictionary , coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.48228875503474306


In [28]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence pour divers nombre de topics

    Paramètres:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Output:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values qui correspond au LDA model avec nombre de topics respectives
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [29]:
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=texts, start=2, limit=50, step=1)

In [30]:
coherence_values

[0.5675437538144065,
 0.6167626643112479,
 0.570294769413644,
 0.4963089067984721,
 0.4774791464095911,
 0.46750418079159317,
 0.48479085091086443,
 0.5485147087910544,
 0.4995925387641414,
 0.45338777299639416,
 0.5066236582942664,
 0.5516341873117073,
 0.5038728596701427,
 0.558795707229534,
 0.5541677139032845,
 0.545428863897199,
 0.57175686248061,
 0.5600872510026104,
 0.5513019214003666,
 0.4486418058383756,
 0.4746677808249935,
 0.44084680685831756,
 0.5187211550973839,
 0.5143532634467862,
 0.5165978932040254,
 0.5012143756518869,
 0.4599751815274425,
 0.5856379932129926,
 0.5575535504297524,
 0.6028718538908231,
 0.5713887364737309,
 0.47355152820059415,
 0.582831167914994,
 0.5523819065360562,
 0.5500815088806846,
 0.672232465843581,
 0.5826130690514146,
 0.6062972995917146,
 0.6213262065522167,
 0.5768060816391354,
 0.5906748152433081,
 0.6189917524814249,
 0.6240722410023175,
 0.5854523777826325,
 0.6487757874706356,
 0.6198025179330728,
 0.6784234638321456,
 0.621878541474

In [33]:
len(coherence_values)

48

In [35]:
type(coherence_values)

list

In [36]:
coherence_values[0]

0.5675437538144065

In [41]:
print('Coherence Score:', coherence_values[46])

Coherence Score: 0.6784234638321456
